# TF-TRT Inference From Saved Model with Tensorflow <= 1.13

This notebook is based on https://github.com/tensorflow/tensorrt/blob/master/tftrt/examples/image-classification/TF-TRT-inference-from-saved-model.ipynb commited on Jun 19, 2019.

## Notebook  Content
1. [Pre-requisite: data and model](#1)
1. [Verifying the orignal FP32 model](#2)
1. [Creating TF-TRT FP32 model](#3)
1. [Creating TF-TRT FP16 model](#4)
1. [Creating TF-TRT INT8 model](#5)
1. [Calibrating TF-TRT INT8 model with raw JPEG images](#6)
 
## Quick start
We will run this demonstration with a Resnet-50 model. The INT8 calibration process requires access to a small but representative sample of real training or valiation data.

We will use the ImageNet dataset that is stored in TFrecords format. Google provides an excellent all-in-one script for downloading and preparing the ImageNet dataset at 

https://github.com/tensorflow/models/blob/master/research/inception/inception/data/download_and_preprocess_imagenet.sh.

<a id="1"></a>
## 1. Pre-requisite: data and model

In [ ]:
import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True

### Data
We verify that the correct Imagenet data folder has been mounted and validation data files of the form `validation-00xxx-of-00128` are available.

In [ ]:
def get_files(data_dir, filename_pattern):
    if data_dir == None:
        return []
    files = tf.gfile.Glob(os.path.join(data_dir, filename_pattern))
    if files == []:
        raise ValueError('Can not find any files in {} with '
                         'pattern "{}"'.format(data_dir, filename_pattern))
    return files

In [ ]:
VALIDATION_DATA_DIR = "PATH-TO-YOUR-IMAGENET-DATASET"

calibration_files = get_files(VALIDATION_DATA_DIR, 'validation*')
print('There are %d calibration files. \n%s\n%s\n...'%(len(calibration_files), calibration_files[0], calibration_files[-1]))

### TF saved model
We will be downloading ResNet-50 trained based on Cloud TPU tutorials.

In [ ]:
!gsutil cp -R gs://sandbox-kathryn-resnet/export/1564508810/* models/resnet/original/00001

In [ ]:
SAVED_MODEL_DIR='{}/models/'.format(os.getcwd())
RESNET_MODEL_DIR=os.path.join(SAVED_MODEL_DIR, 'resnet')
ORIGINAL_MODEL_DIR=os.path.join(RESNET_MODEL_DIR, 'original', '00001')

### Helper functions
We define a few helper functions to read and preprocess Imagenet data from TFRecord files. 

In [ ]:
example = next(tf.python_io.tf_record_iterator("gs://sandbox-kathryn-data/imagenet/validation-00000-of-00128"))

In [ ]:
def deserialize_image_record(record):
    keys_to_features = {
        'image/encoded': tf.FixedLenFeature((), tf.string, ''),
        'image/class/label': tf.FixedLenFeature([], tf.int64, -1),
    }

    with tf.name_scope('deserialize_image_record'):
        parsed = tf.parse_single_example(record, keys_to_features)
        image_bytes = tf.reshape(parsed['image/encoded'], shape=[])
        label = tf.cast(tf.reshape(parsed['image/class/label'], shape=[]), dtype=tf.int32)
        
        # 'image/class/label' is encoded as an integer from 1 to num_label_classes
        # In order to generate the correct one-hot label vector from this number,
        # we subtract the number by 1 to make it in [0, num_label_classes).
        label -= 1
        return image_bytes, label

In [ ]:
image_size=224
CROP_PADDING=32

def preprocess(record):
    image_bytes, label = deserialize_image_record(record)
    
    shape = tf.image.extract_jpeg_shape(image_bytes)
    image_height = shape[0]
    image_width = shape[1]
    
    padded_center_crop_size = tf.cast(
        ((image_size / (image_size + CROP_PADDING)) *
         tf.cast(tf.minimum(image_height, image_width), tf.float32)),
        tf.int32)

    offset_height = ((image_height - padded_center_crop_size) + 1) // 2
    offset_width = ((image_width - padded_center_crop_size) + 1) // 2
    crop_window = tf.stack([offset_height, offset_width,
                            padded_center_crop_size, padded_center_crop_size])
    image = tf.image.decode_and_crop_jpeg(image_bytes, crop_window, channels=3)
    image = tf.image.resize_bicubic([image], [image_size, image_size])[0]
    image = tf.reshape(image, [image_size, image_size, 3])
    image = tf.image.convert_image_dtype(image, tf.float32)
    
    return image, label

In [ ]:
# non-use
def deserialize_image_record(record):
    feature_map = {
        'image/encoded':          tf.FixedLenFeature([ ], tf.string, ''),
        'image/class/label':      tf.FixedLenFeature([1], tf.int64,  -1),
        'image/class/text':       tf.FixedLenFeature([ ], tf.string, ''),
        'image/object/bbox/xmin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymin': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/xmax': tf.VarLenFeature(dtype=tf.float32),
        'image/object/bbox/ymax': tf.VarLenFeature(dtype=tf.float32)
    }
    with tf.name_scope('deserialize_image_record'):
        obj = tf.parse_single_example(record, feature_map)
        imgdata = obj['image/encoded']
        label   = tf.cast(obj['image/class/label'], tf.int32)
        bbox    = tf.stack([obj['image/object/bbox/%s'%x].values
                            for x in ['ymin', 'xmin', 'ymax', 'xmax']])
        bbox = tf.transpose(tf.expand_dims(bbox, 0), [0,2,1])
        text    = obj['image/class/text']
        return imgdata, label, bbox, text

In [ ]:
# non-use
from preprocessing import vgg_preprocessing
def preprocess(record):
    # Parse TFRecord
    imgdata, label, bbox, text = deserialize_image_record(record)
    #label -= 1 # Change to 0-based if not using background class
    try:    image = tf.image.decode_jpeg(imgdata, channels=3, fancy_upscaling=False, dct_method='INTEGER_FAST')
    except: image = tf.image.decode_png(imgdata, channels=3)

    image = vgg_preprocessing.preprocess_image(image, 224, 224, is_training=False)
    return image, label

In [ ]:
#Define some global variables
BATCH_SIZE = 64

dataset = tf.data.TFRecordDataset(calibration_files)    
dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=preprocess, batch_size=BATCH_SIZE, num_parallel_calls=20))

<a id="2"></a>
## 2. Verifying the orignal FP32 model
We demonstrate the conversion process with a Resnet-50 v1 model. First, we inspect the original Tensorflow model.

We employ `saved_model_cli` to inspect the inputs and outputs of the model.

In [ ]:
!saved_model_cli show --all --dir $ORIGINAL_MODEL_DIR

This give us information on the input and output tensors as `input_tensor:0` and `softmax_tensor:0` respectively. Also note that the number of output classes here is 1001 instead of 1000 Imagenet classes. This is because the network was trained with an extra background class. 

In [ ]:
INPUT_TENSOR = 'input_tensor:0'
OUTPUT_TENSOR = 'softmax_tensor:0'

Next, we define a function to read in a saved mode, measuring its speed and accuracy on the validation data.

In [ ]:
def benchmark_saved_model(SAVED_MODEL_DIR, dataset=dataset, BATCH_SIZE=64):
    with tf.Session(graph=tf.Graph(), config=config) as sess:
        # prepare dataset iterator
        iterator = dataset.make_one_shot_iterator()
        next_element = iterator.get_next()

        tf.saved_model.loader.load(
            sess, [tf.saved_model.tag_constants.SERVING], SAVED_MODEL_DIR)

        print('Warming up for 50 batches...')
        for _ in range (50):
            sess.run(OUTPUT_TENSOR, feed_dict={INPUT_TENSOR: sess.run(next_element)[0]})

        print('Benchmarking inference engine...')
        num_hits = 0
        num_predict = 0
        start_time = time.time()
        try:
            while True:        
                image_data = sess.run(next_element)    
                img = image_data[0]
                label = image_data[1].squeeze()
                output = sess.run([OUTPUT_TENSOR], feed_dict={INPUT_TENSOR: img})            
                prediction = np.argmax(output[0], axis=1)
                num_hits += np.sum(prediction == label)
                num_predict += len(prediction)
        except tf.errors.OutOfRangeError as e:
            pass

        print('Accuracy: %.2f%%'%(100*num_hits/num_predict))
        print('Inference speed: %.2f samples/s'%(num_predict/(time.time()-start_time)))

In [ ]:
benchmark_saved_model(ORIGINAL_MODEL_DIR, dataset=dataset, BATCH_SIZE=BATCH_SIZE)

<a id="3"></a>
## 3. Creating TF-TRT FP32 model

Next, we convert the naitive TF FP32 model to TF-TRT FP32, then verify model accuracy and inference speed.

In [ ]:
FP32_SAVED_MODEL_DIR = os.path.join(RESNET_MODEL_DIR, "FP32", "00001")
!rm -rf $FP32_SAVED_MODEL_DIR

#Now we create the TFTRT FP32 engine
_ = trt.create_inference_graph(
    input_graph_def=None,
    outputs=None,
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=ORIGINAL_MODEL_DIR,
    output_saved_model_dir=FP32_SAVED_MODEL_DIR,
    precision_mode="FP32")

In [ ]:
benchmark_saved_model(FP32_SAVED_MODEL_DIR, dataset=dataset, BATCH_SIZE=BATCH_SIZE)

<a id="4"></a>
## 4. Creating TF-TRT FP16 model

Next, we convert the naitive TF FP32 model to TF-TRT FP16, then verify model accuracy and inference speed.

In [ ]:
FP16_SAVED_MODEL_DIR = os.path.join(RESNET_MODEL_DIR, "FP16", "00001")
!rm -rf $FP16_SAVED_MODEL_DIR

#Now we create the TFTRT FP16 engine
_ = trt.create_inference_graph(
    input_graph_def=None,
    outputs=None,
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=ORIGINAL_MODEL_DIR,
    output_saved_model_dir=FP16_SAVED_MODEL_DIR,
    precision_mode="FP16")

In [ ]:
benchmark_saved_model(FP16_SAVED_MODEL_DIR, dataset=dataset, BATCH_SIZE=BATCH_SIZE)

<a id="5"></a>
## 5. Creating TF-TRT INT8 model

Creating TF-TRT INT8 inference model requires two steps:

- Step 1: creating the calibration graph, and run some training data through that graph for INT-8 calibration.

- Step 2: converting the calibration graph to the TF-TRT INT8 inference engine

### Step 1: Creating the calibration graph

In [ ]:
#Now we create the TFTRT INT8 calibration graph
trt_int8_calib_graph = trt.create_inference_graph(
    input_graph_def=None,
    outputs=[OUTPUT_TENSOR],
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=ORIGINAL_MODEL_DIR,    
    precision_mode="INT8")

In [ ]:
#Then calibrate it with 10 batches of examples
N_runs=10
with tf.Session(graph=tf.Graph(), config=config) as sess:
    print('Preparing calibration data...')
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading INT8 calibration graph...')
    output_node = tf.import_graph_def(
        trt_int8_calib_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    print('Calibrate model on calibration data...')    
    for _ in range(N_runs):
        sess.run(output_node, feed_dict={INPUT_TENSOR: sess.run(next_element)[0]})            

### Step 2: Converting the calibration graph to inference graph

Now we convert the INT8 calibration graph to the final TF-TRT INT8 inference engine, then save this engine to a *saved model*, ready to be served elsewhere.

In [ ]:
#Create Int8 inference model from the calibration graph and write to a saved session
print('Creating TF-TRT INT8 inference engine...')
trt_int8_calibrated_graph=trt.calib_graph_to_infer_graph(trt_int8_calib_graph)

In [ ]:
#set a directory to write the saved model
INT8_SAVED_MODEL_DIR =  os.path.join(RESNET_MODEL_DIR, "INT8/00001")
!rm -rf $INT8_SAVED_MODEL_DIR

with tf.Session(graph=tf.Graph()) as sess:
    print('Loading TF-TRT INT8 inference engine...')
    output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    #Save model for serving
    print('Saving INT8 model to %s'%INT8_SAVED_MODEL_DIR)
    tf.saved_model.simple_save(
        session=sess,
        export_dir=INT8_SAVED_MODEL_DIR,
        inputs={"input":tf.get_default_graph().get_tensor_by_name(INPUT_TENSOR)},
        outputs={"softmax":tf.get_default_graph().get_tensor_by_name(OUTPUT_TENSOR),
                 "classes":tf.get_default_graph().get_tensor_by_name("ArgMax:0")},
        legacy_init_op=None
     )    

### Benchmarking INT8 saved model

Finally we reload and verify the accuracy and performance of the INT8 saved model from disk.

In [ ]:
benchmark_saved_model(INT8_SAVED_MODEL_DIR)

In [ ]:
!saved_model_cli show --all --dir $INT8_SAVED_MODEL_DIR

<a id="6"></a>
## 6. Calibrating TF-TRT INT8 model with raw JPEG images

As an alternative to taking data in TFRecords format, in this section, we demonstrate the process of calibrating TFTRT INT-8 model from a directory of raw JPEG images. We asume that raw images have been mounted to the directory `/data/Calibration_data`.

As a rule of thumb, calibration data should be a small but representative set of images that is similar to what is expected in deployment. Empirically, for common network architectures trained on imagenet data, calibration data of size 500-1000 provide good accuracy. As such, a good strategy for a dataset such as imagenet is to choose one sample from each class. 

In [ ]:
data_directory = "/data/Calibration_data"
calibration_files = [os.path.join(path, name) for path, _, files in os.walk(data_directory) for name in files]
print('There are %d calibration files. \n%s\n%s\n...'%(len(calibration_files), calibration_files[0], calibration_files[-1]))

We define a helper function to read and preprocess image from JPEG file.

In [ ]:
def parse_file(filepath):
    image = tf.read_file(filepath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.reshape(image, [224, 224, 3] is_training=False)
    return image

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(data_files)
dataset = dataset.apply(tf.contrib.data.map_and_batch(map_func=parse_file, batch_size=BATCH_SIZE, num_parallel_calls=20))
dataset = dataset.repeat(count=1)

Next, we proceed with the two-stage process of creating and calibrating TFTRT INT8 model.

### Step 1: Creating the calibration graph

In [ ]:
#Now we create the TFTRT INT8 calibration graph
trt_int8_calib_graph = trt.create_inference_graph(
    input_graph_def=None,
    outputs=[OUTPUT_TENSOR],
    max_batch_size=BATCH_SIZE,
    input_saved_model_dir=SAVED_MODEL_DIR,    
    precision_mode="INT8")

#Then calibrate it with 10 batches of examples
N_runs=10
with tf.Session(graph=tf.Graph(), config=config) as sess:
    print('Preparing calibration data...')
    iterator = dataset.make_one_shot_iterator()
    next_element = iterator.get_next()

    print('Loading INT8 calibration graph...')
    output_node = tf.import_graph_def(
        trt_int8_calib_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    print('Calibrate model on calibration data...')    
    for _ in range(N_runs):
        sess.run(output_node, feed_dict={INPUT_TENSOR: sess.run(next_element)})            

### Step 2: Converting the calibration graph to inference graph

In [ ]:
#Create Int8 inference model from the calibration graph and write to a saved session
print('Creating TF-TRT INT8 inference engine...')
trt_int8_calibrated_graph=trt.calib_graph_to_infer_graph(trt_int8_calib_graph)

#set a directory to write the saved model
INT8_SAVED_MODEL_DIR =  SAVED_MODEL_DIR + "_TFTRT_INT8_JPEG/1"
!rm -rf $INT8_SAVED_MODEL_DIR

with tf.Session(graph=tf.Graph()) as sess:
    print('Loading TF-TRT INT8 inference engine...')
    output_node = tf.import_graph_def(
        trt_int8_calibrated_graph,
        return_elements=[OUTPUT_TENSOR],
        name='')

    #Save model for serving
    print('Saving INT8 model to %s'%INT8_SAVED_MODEL_DIR)
    tf.saved_model.simple_save(
        session=sess,
        export_dir=INT8_SAVED_MODEL_DIR,
        inputs={"input":tf.get_default_graph().get_tensor_by_name(INPUT_TENSOR)},
        outputs={"softmax":tf.get_default_graph().get_tensor_by_name(OUTPUT_TENSOR),
                 "classes":tf.get_default_graph().get_tensor_by_name("ArgMax:0")},
        legacy_init_op=None
     )    

As before, we can benchmark the speed and accuracy of the resulting model.

In [ ]:
benchmark_saved_model(INT8_SAVED_MODEL_DIR)

## Conclusion
In this notebook, we have demonstrated the process of creating TF-TRT inference model from an original TF FP32 *saved model*. In every case, we have also verified the accuracy and speed to the resulting model. 
